In [1]:
import os
import time
import numpy as np
import numpy.random as npr
import re
import nltk
import matplotlib.pyplot as plt
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from tqdm import tqdm
from scipy.special import digamma, loggamma

from typing import List, Dict, Tuple, Set
import logging

In [2]:
def load_vocabulary(vocab_file: str) -> Dict[str, int]:
    stop_words = set(stopwords.words("english"))
    vocab = {}
    with open(vocab_file, "r") as file:
        for idx, word in enumerate(file):
            if word not in stop_words:
                vocab[word.strip()] = idx

    return vocab

def preprocess_text(text: str) -> List[str]:
    stop_words = set(stopwords.words("english"))
    text = text.lower()
    words = re.findall(r"\b\w+\b", text)
    words = [word for word in words if word not in stop_words]

    return words

def parse_documents(corpus_file: str) -> List[str]:
    with open(corpus_file, "r") as file:
        raw_text = file.read()
    
    pattern = r"<TEXT>\s*(.*?)\s*</TEXT>"
    documents = re.findall(pattern, raw_text, re.DOTALL)

    return documents

def create_document_term_matrix(documents: List[str], vocab: Dict[str, int]) -> np.ndarray:
    N = len(documents)
    V = len(vocab)
    doc_term_matrix = np.zeros((N, V), dtype=int)

    for doc_idx, doc in enumerate(documents):
        words = preprocess_text(doc)
        for word in words:
            if word in vocab:
                vocab_idx = vocab[word]
                doc_term_matrix[doc_idx, vocab_idx] += 1
    
    return doc_term_matrix

documents = parse_documents("ap.txt")
vocab = load_vocabulary("vocab.txt")
dtm = create_document_term_matrix(documents=documents, vocab=vocab)

In [3]:
K = 225
N, V = dtm.shape
LAMBDA = np.random.gamma(100., 1./100., (K, V))
GAMMA = np.ones((N, K))
PHI = [np.ones((len(doc), K)) / K for doc in dtm]

MemoryError: Unable to allocate 18.0 MiB for an array with shape (10473, 225) and data type float64

In [8]:
doc = dtm[0]
nonzero_idxs = np.sum(doc > 0)
nonzero_idxs

np.int64(177)